In [2]:
#Import libraries
from pyhive import presto
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import h3
from shapely.geometry import Polygon, Point
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 500)

In [3]:
#Define presto credentials
presto_host = 'presto.processing.yoda.run'
presto_port = '80'
username = 'aditya.bhattar@rapido.bike'

#Create connection to presto host
connection = presto.connect(presto_host,presto_port,username = username)

In [4]:
#Query data from order_logs_snapshot

q = """
select order_id, accept_to_pickup_distance, captain_location_hex_8, captain_location_latitude,
captain_location_longitude, city_name, customer_location_hex_8, customer_location_latitude, 
customer_location_longitude, hhmmss, hour, quarter_hour, service_obj_service_name, time_bucket, weekday, yyyymmdd,
order_status, cancel_reason
from orders.order_logs_snapshot
where yyyymmdd >= '20220919'
and yyyymmdd <= '20220925'
and city_name in ('Bangalore')
and service_obj_service_name = 'Link'
"""

#Load data into pandas table
df_copy = pd.read_sql(q, connection)

In [5]:
#Create a copy of the df retrived from presto
df = df_copy.copy()

In [6]:
#View the dataset
df.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason
0,63297ec0ae81787578f4d623,0.507,8861892ec3fffff,12.977918,77.639038,Bangalore,8861892533fffff,12.958934,77.637807,142008,14,1415,Link,14:00,2,20220920,dropped,Asked to cancel & take offline ride
1,63297edf853c020ac7973545,1.511,88618924b1fffff,12.905964,77.558372,Bangalore,88618925d3fffff,12.916800,77.586797,142039,14,1415,Link,14:00,2,20220920,dropped,Unsafe Pickup area
2,6329800549a7d319a068e4a3,1.970,8861892e25fffff,13.021454,77.699936,Bangalore,8861892c09fffff,13.037684,77.663254,142533,14,1415,Link,14:00,2,20220920,dropped,None
3,632980c57d42681b1c45aca8,1.851,886016968bfffff,13.087599,77.509918,Bangalore,886016968bfffff,13.087613,77.509850,142845,14,1415,Link,14:00,2,20220920,customerCancelled,order cancelled before rider accepted
4,632980c9853c020ac79739bf,0.213,8860145a19fffff,12.925041,77.526558,Bangalore,8860145b25fffff,13.024989,77.585229,142849,14,1415,Link,14:00,2,20220920,dropped,None


In [7]:
#Make columns for second and minute and date
df['second'] = df['hhmmss'].apply(lambda x: x[4:])
df['minute'] = df['hhmmss'].apply(lambda x: x[2:4])
df['date'] = df['yyyymmdd'].apply(lambda x: x[6:])
df.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,second,minute,date
0,63297ec0ae81787578f4d623,0.507,8861892ec3fffff,12.977918,77.639038,Bangalore,8861892533fffff,12.958934,77.637807,142008,14,1415,Link,14:00,2,20220920,dropped,Asked to cancel & take offline ride,08,20,20
1,63297edf853c020ac7973545,1.511,88618924b1fffff,12.905964,77.558372,Bangalore,88618925d3fffff,12.916800,77.586797,142039,14,1415,Link,14:00,2,20220920,dropped,Unsafe Pickup area,39,20,20
2,6329800549a7d319a068e4a3,1.970,8861892e25fffff,13.021454,77.699936,Bangalore,8861892c09fffff,13.037684,77.663254,142533,14,1415,Link,14:00,2,20220920,dropped,None,33,25,20
3,632980c57d42681b1c45aca8,1.851,886016968bfffff,13.087599,77.509918,Bangalore,886016968bfffff,13.087613,77.509850,142845,14,1415,Link,14:00,2,20220920,customerCancelled,order cancelled before rider accepted,45,28,20
4,632980c9853c020ac79739bf,0.213,8860145a19fffff,12.925041,77.526558,Bangalore,8860145b25fffff,13.024989,77.585229,142849,14,1415,Link,14:00,2,20220920,dropped,None,49,28,20


In [8]:
#Create 15-sec batches and 20-sec batches
df['second'] = df['second'].astype(float)
df['minute'] = df['minute'].astype(float)
df['20_sec_batch'] = pd.cut(df['second'], bins = [-1, 20, 40, 61], labels = ['0-20', '20-40', '40-60'])
df['count'] = 1
df.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,second,minute,date,20_sec_batch,count
0,63297ec0ae81787578f4d623,0.507,8861892ec3fffff,12.977918,77.639038,Bangalore,8861892533fffff,12.958934,77.637807,142008,14,1415,Link,14:00,2,20220920,dropped,Asked to cancel & take offline ride,8.0,20.0,20,0-20,1
1,63297edf853c020ac7973545,1.511,88618924b1fffff,12.905964,77.558372,Bangalore,88618925d3fffff,12.916800,77.586797,142039,14,1415,Link,14:00,2,20220920,dropped,Unsafe Pickup area,39.0,20.0,20,20-40,1
2,6329800549a7d319a068e4a3,1.970,8861892e25fffff,13.021454,77.699936,Bangalore,8861892c09fffff,13.037684,77.663254,142533,14,1415,Link,14:00,2,20220920,dropped,None,33.0,25.0,20,20-40,1
3,632980c57d42681b1c45aca8,1.851,886016968bfffff,13.087599,77.509918,Bangalore,886016968bfffff,13.087613,77.509850,142845,14,1415,Link,14:00,2,20220920,customerCancelled,order cancelled before rider accepted,45.0,28.0,20,40-60,1
4,632980c9853c020ac79739bf,0.213,8860145a19fffff,12.925041,77.526558,Bangalore,8860145b25fffff,13.024989,77.585229,142849,14,1415,Link,14:00,2,20220920,dropped,None,49.0,28.0,20,40-60,1


In [9]:
#Add column to identify the hexes
bangalore_zone1 = pd.read_csv('/Users/rapido/Desktop/batching_demand/bangalore_location_data_level1_demand_zone1.csv')
bangalore_zone2 = pd.read_csv('/Users/rapido/Desktop/batching_demand/bangalore_location_data_level1_demand_zone2.csv')

bangalore_zone1_hexes = list(bangalore_zone1['customer_location_hex_8'].unique())
bangalore_zone2_hexes = list(bangalore_zone2['customer_location_hex_8'].unique())

df['zone_name'] = np.where(df['customer_location_hex_8'].isin(bangalore_zone1_hexes), 'zone1',
     np.where(df['customer_location_hex_8'].isin(bangalore_zone2_hexes), 'zone2', 'rest_of_bangalore'))
df.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,second,minute,date,20_sec_batch,count,zone_name
0,63297ec0ae81787578f4d623,0.507,8861892ec3fffff,12.977918,77.639038,Bangalore,8861892533fffff,12.958934,77.637807,142008,14,1415,Link,14:00,2,20220920,dropped,Asked to cancel & take offline ride,8.0,20.0,20,0-20,1,rest_of_bangalore
1,63297edf853c020ac7973545,1.511,88618924b1fffff,12.905964,77.558372,Bangalore,88618925d3fffff,12.916800,77.586797,142039,14,1415,Link,14:00,2,20220920,dropped,Unsafe Pickup area,39.0,20.0,20,20-40,1,rest_of_bangalore
2,6329800549a7d319a068e4a3,1.970,8861892e25fffff,13.021454,77.699936,Bangalore,8861892c09fffff,13.037684,77.663254,142533,14,1415,Link,14:00,2,20220920,dropped,None,33.0,25.0,20,20-40,1,rest_of_bangalore
3,632980c57d42681b1c45aca8,1.851,886016968bfffff,13.087599,77.509918,Bangalore,886016968bfffff,13.087613,77.509850,142845,14,1415,Link,14:00,2,20220920,customerCancelled,order cancelled before rider accepted,45.0,28.0,20,40-60,1,rest_of_bangalore
4,632980c9853c020ac79739bf,0.213,8860145a19fffff,12.925041,77.526558,Bangalore,8860145b25fffff,13.024989,77.585229,142849,14,1415,Link,14:00,2,20220920,dropped,None,49.0,28.0,20,40-60,1,rest_of_bangalore


In [10]:
#Create a column to identify OCARA, COBRA and COBRM
df['new_cancel_reason'] = np.where(df['order_status'] == 'customerCancelled', np.where(df['cancel_reason'] == 'order cancelled before rider accepted', 'COBRA',
    np.where(df['cancel_reason'] == 'Order cancelled before rider was mapped', 'COBRM', 'OCARA')), df['order_status'])
df.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,second,minute,date,20_sec_batch,count,zone_name,new_cancel_reason
0,63297ec0ae81787578f4d623,0.507,8861892ec3fffff,12.977918,77.639038,Bangalore,8861892533fffff,12.958934,77.637807,142008,14,1415,Link,14:00,2,20220920,dropped,Asked to cancel & take offline ride,8.0,20.0,20,0-20,1,rest_of_bangalore,dropped
1,63297edf853c020ac7973545,1.511,88618924b1fffff,12.905964,77.558372,Bangalore,88618925d3fffff,12.916800,77.586797,142039,14,1415,Link,14:00,2,20220920,dropped,Unsafe Pickup area,39.0,20.0,20,20-40,1,rest_of_bangalore,dropped
2,6329800549a7d319a068e4a3,1.970,8861892e25fffff,13.021454,77.699936,Bangalore,8861892c09fffff,13.037684,77.663254,142533,14,1415,Link,14:00,2,20220920,dropped,None,33.0,25.0,20,20-40,1,rest_of_bangalore,dropped
3,632980c57d42681b1c45aca8,1.851,886016968bfffff,13.087599,77.509918,Bangalore,886016968bfffff,13.087613,77.509850,142845,14,1415,Link,14:00,2,20220920,customerCancelled,order cancelled before rider accepted,45.0,28.0,20,40-60,1,rest_of_bangalore,COBRA
4,632980c9853c020ac79739bf,0.213,8860145a19fffff,12.925041,77.526558,Bangalore,8860145b25fffff,13.024989,77.585229,142849,14,1415,Link,14:00,2,20220920,dropped,None,49.0,28.0,20,40-60,1,rest_of_bangalore,dropped


In [11]:
#Filter the df on the 3 zones
df_zone1 = df[df['zone_name'] == 'zone1']
df_zone2 = df[df['zone_name'] == 'zone2']
df_zone_rest = df[df['zone_name'] == 'rest_of_bangalore']

In [20]:
#Check the end order state for all orders occurring in a 20_sec_batch for zone1
mistiming_pivot_zone1 = pd.pivot_table(data = df_zone1, index = ['date', 'hour', 'minute', '20_sec_batch'], columns = 'new_cancel_reason', values = 'count',
aggfunc='sum', fill_value=0).reset_index().rename_axis(None, axis=1)

#Get the percentage for all order types
mistiming_pivot_zone1['total'] = mistiming_pivot_zone1[['COBRA', 'COBRM', 'OCARA', 'aborted', 'dropped', 'expired', 'new']].sum(axis = 1)
mistiming_pivot_zone1['dropped_%'] = round(mistiming_pivot_zone1['dropped']/mistiming_pivot_zone1['total'], 4)
mistiming_pivot_zone1['COBRA_%'] = round(mistiming_pivot_zone1['COBRA']/mistiming_pivot_zone1['total'], 4)
mistiming_pivot_zone1['COBRM_%'] = round(mistiming_pivot_zone1['COBRM']/mistiming_pivot_zone1['total'], 4)
mistiming_pivot_zone1['OCARA_%'] = round(mistiming_pivot_zone1['OCARA']/mistiming_pivot_zone1['total'], 4)
mistiming_pivot_zone1['expiry_%'] = round(mistiming_pivot_zone1['expired']/mistiming_pivot_zone1['total'], 4)

#Get the average of each percentage by hour
mistiming_grouped_zone1 = mistiming_pivot_zone1.groupby(['date', 'hour'])['dropped_%', 'COBRA_%', 'COBRM_%', 'OCARA_%', 'expiry_%'].mean().reset_index()
mistiming_grouped_zone1.head()

,date,hour,dropped_%,COBRA_%,COBRM_%,OCARA_%,expiry_%
0,19,00,0.255298,0.349722,0.002032,0.366215,0.026727
1,19,01,0.306496,0.353106,0.000000,0.311441,0.026836
2,19,02,0.343254,0.346230,0.000000,0.280754,0.017857
3,19,03,0.265152,0.343435,0.000000,0.340909,0.045455
4,19,04,0.161984,0.521536,0.000000,0.215356,0.101122


In [21]:
#Check the end order state for all orders occurring in a 20_sec_batch for zone2
mistiming_pivot_zone2 = pd.pivot_table(data = df_zone2, index = ['date', 'hour', 'minute', '20_sec_batch'], columns = 'new_cancel_reason', values = 'count',
aggfunc='sum', fill_value=0).reset_index().rename_axis(None, axis=1)

#Get the percentage for all order types
mistiming_pivot_zone2['total'] = mistiming_pivot_zone2[['COBRA', 'COBRM', 'OCARA', 'aborted', 'dropped', 'expired', 'new']].sum(axis = 1)
mistiming_pivot_zone2['dropped_%'] = round(mistiming_pivot_zone2['dropped']/mistiming_pivot_zone2['total'], 4)
mistiming_pivot_zone2['COBRA_%'] = round(mistiming_pivot_zone2['COBRA']/mistiming_pivot_zone2['total'], 4)
mistiming_pivot_zone2['COBRM_%'] = round(mistiming_pivot_zone2['COBRM']/mistiming_pivot_zone2['total'], 4)
mistiming_pivot_zone2['OCARA_%'] = round(mistiming_pivot_zone2['OCARA']/mistiming_pivot_zone2['total'], 4)
mistiming_pivot_zone2['expiry_%'] = round(mistiming_pivot_zone2['expired']/mistiming_pivot_zone2['total'], 4)

#Get the average of each percentage by hour
mistiming_grouped_zone2 = mistiming_pivot_zone2.groupby(['date', 'hour'])['dropped_%', 'COBRA_%', 'COBRM_%', 'OCARA_%', 'expiry_%'].mean().reset_index()
mistiming_grouped_zone2.head()

,date,hour,dropped_%,COBRA_%,COBRM_%,OCARA_%,expiry_%
0,19,00,0.462962,0.236625,0.0,0.283949,0.016460
1,19,01,0.341463,0.296748,0.0,0.349593,0.012195
2,19,02,0.413793,0.362069,0.0,0.224138,0.000000
3,19,03,0.406250,0.406250,0.0,0.187500,0.000000
4,19,04,0.264706,0.397059,0.0,0.294118,0.044118


In [22]:
#Check the end order state for all orders occurring in a 20_sec_batch for zone2
mistiming_pivot_rest = pd.pivot_table(data = df_zone_rest, index = ['date', 'hour', 'minute', '20_sec_batch'], columns = 'new_cancel_reason', values = 'count',
aggfunc='sum', fill_value=0).reset_index().rename_axis(None, axis=1)

#Get the percentage for all order types
mistiming_pivot_rest['total'] = mistiming_pivot_rest[['COBRA', 'COBRM', 'OCARA', 'aborted', 'dropped', 'expired', 'new']].sum(axis = 1)
mistiming_pivot_rest['dropped_%'] = round(mistiming_pivot_rest['dropped']/mistiming_pivot_rest['total'], 4)
mistiming_pivot_rest['COBRA_%'] = round(mistiming_pivot_rest['COBRA']/mistiming_pivot_rest['total'], 4)
mistiming_pivot_rest['COBRM_%'] = round(mistiming_pivot_rest['COBRM']/mistiming_pivot_rest['total'], 4)
mistiming_pivot_rest['OCARA_%'] = round(mistiming_pivot_rest['OCARA']/mistiming_pivot_rest['total'], 4)
mistiming_pivot_rest['expiry_%'] = round(mistiming_pivot_rest['expired']/mistiming_pivot_rest['total'], 4)

#Get the average of each percentage by hour
mistiming_grouped_rest = mistiming_pivot_rest.groupby(['date', 'hour'])['dropped_%', 'COBRA_%', 'COBRM_%', 'OCARA_%', 'expiry_%'].mean().reset_index()
mistiming_grouped_rest.head()

,date,hour,dropped_%,COBRA_%,COBRM_%,OCARA_%,expiry_%
0,19,00,0.286156,0.388919,0.009083,0.223596,0.090559
1,19,01,0.207428,0.365292,0.025205,0.270278,0.131801
2,19,02,0.209502,0.359404,0.024171,0.272416,0.133039
3,19,03,0.107611,0.399558,0.054423,0.200115,0.236741
4,19,04,0.165879,0.395172,0.056234,0.175602,0.207106


In [23]:
#Save the results
mistiming_grouped_zone1.to_csv('mistiming_bangalore_zone1.csv', index = False)
mistiming_grouped_zone2.to_csv('mistiming_bangalore_zone2.csv', index = False)
mistiming_grouped_rest.to_csv('mistiming_bangalore_rest.csv', index = False)